In [1]:
import tensorflow as tf
import utils

In [2]:
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

In [3]:
class Conv2D(tf.keras.layers.Conv2D):
    def __init__(self,f,k,a,b,**kwdarg):
        super().__init__(
            filters=f,
            kernel_size=k,
            activation=a,
            use_bias=b,
            strides=1,
            padding='valid',
            **kwdarg)
    def call(self,inp,training):
        x = super().call(inp)
        return x

In [4]:
class DepthwiseConv2D(tf.keras.layers.DepthwiseConv2D):
    def __init__(self,m,k,a,b,**kwdarg):
        super().__init__(
            depth_multiplier=m,
            kernel_size=k,
            activation=a,
            use_bias=b,
            strides=1,
            padding='valid',
            **kwdarg)
    def call(self,inp,training):
        x = super().call(inp)
        return x

In [5]:
class DepthControl(tf.keras.layers.Layer):
    def __init__(self,r,a,b,groups=1):
        super().__init__()
        self.r = 2**r
        self.a = a
        self.b = b
        self.groups = groups
    def build(self,input_shape):
        self.conv = Conv2D(
            f=int(input_shape[-1]*self.r),
            k=1,
            a=self.a,
            b=self.b,
            groups=self.groups
        )
    def call(self,inp,training):
        x = self.conv(inp,training=training)
        return x

In [6]:
class SubBlock(tf.keras.layers.Layer):
    def __init__(self,vision_layer,nins):
        super().__init__()
        self.vision_layer = vision_layer
        self.main = tf.keras.Sequential([DepthControl(r=r,a=mish,b=True) for r in nins]+
                                        [DepthControl(r=-r,a=mish,b=True) for r in nins[::-1]])
        self.mix = DepthControl(r=-1,a=mish,b=False)
    def call(self,inp,training):
        x1 = self.vision_layer(inp,training=training)
        x2 = self.main(x1,training=training)
        x = tf.concat([x1,x2],axis=-1)
        x = self.mix(x,training=training)
        return x

In [7]:
class GroupSubBlock(tf.keras.layers.Layer):
    def __init__(self,vision_layer,groups,nins):
        super().__init__()
        self.vision_layer = vision_layer
        self.main = tf.keras.Sequential([DepthControl(r=r,a=mish,b=True,groups=groups) for r in nins]+
                                        [DepthControl(r=-r,a=mish,b=True,groups=groups) for r in nins[::-1]])
    def call(self,inp,training):
        x1 = self.vision_layer(inp,training=training)
        x2 = self.main(x1,training=training)
        x = x1 + x2
        return x

In [8]:
class Contract(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        self.downsample = tf.keras.layers.AveragePooling2D(pool_size=2, strides=2, padding='valid')
        self.increase = DepthControl(r=1,a=None,b=False)
    def call(self,inp,training):
        x = self.downsample(inp,training=training)
        x = self.increase(x,training=training)
        return x

In [9]:
class Model(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.block1 = tf.keras.Sequential([ #B,40,240,1
            SubBlock(DepthwiseConv2D(m=8,k=3,a=None,b=False),[1,1]),  #B.38,238,8
            SubBlock(Conv2D(f=8,k=3,a=mish,b=True),[1,1]), #B.36,236,8
            Contract()]) #B,18,118,16
        self.block2 = tf.keras.Sequential([
            SubBlock(Conv2D(f=16,k=3,a=mish,b=True),[1,0,0]), #B,16,116,16
            Contract()]) #B,8,58,32
        self.block3 = tf.keras.Sequential([
            SubBlock(Conv2D(f=32,k=3,a=mish,b=True),[1,0,0]), #B,6,56,32
            SubBlock(Conv2D(f=32,k=3,a=mish,b=True),[1,0,0]), #B,4,54,,32
            Contract()]) #B,3,27,64
        self.block4 = tf.keras.Sequential([
            SubBlock(Conv2D(f=64,k=2,a=mish,b=True),[-1,0,0]), #B,1,25,64
#             tf.keras.layers.Lambda(lambda x: tf.tile(x,[1,1,1,12])) #B,1,25,12*64
        ])
        self.block5 = tf.keras.Sequential([
            SubBlock(Conv2D(f=128,k=(1,2),a=mish,b=True),[0,0]), #25
            SubBlock(Conv2D(f=128,k=(1,2),a=mish,b=True),[0,0]), #24
            SubBlock(Conv2D(f=128,k=(1,2),a=mish,b=True),[0,0]), #23
            tf.keras.layers.AveragePooling2D(pool_size=(1,3), strides=(1,2), padding='valid'), #11
            SubBlock(Conv2D(f=256,k=(1,2),a=mish,b=True),[0,0]), #10
            SubBlock(Conv2D(f=256,k=(1,2),a=mish,b=True),[0,0]), #9
            tf.keras.layers.AveragePooling2D(pool_size=(1,3), strides=(1,2), padding='valid'), #4
            SubBlock(Conv2D(f=512,k=(1,2),a=mish,b=True),[0,0]), #3
            SubBlock(Conv2D(f=512,k=(1,2),a=mish,b=True),[0,0]), #2
            SubBlock(Conv2D(f=1024,k=(1,2),a=mish,b=True),[0,0]), #B,1,1,1024
            
            tf.keras.layers.Lambda(lambda x: tf.squeeze(x,axis=[1,2])), #B,1024
#             tf.keras.layers.Reshape([1,12,-1])#B,1,12,344
        ])
        self.output_layer = tf.keras.Sequential([
            tf.keras.layers.Dense(2048,activation=mish,use_bias=True),
            tf.keras.layers.Dense(1024,activation=mish,use_bias=True),
            tf.keras.layers.Dense(512,activation=mish,use_bias=True),
            tf.keras.layers.Dense(12*36,activation=None,use_bias=False),
            tf.keras.layers.Reshape([12,36])#B,12,36
        ])
        
    def call(self,image,training):
        x = self.block1(image,training=training)
        x = self.block2(x,training=training)
        x = self.block3(x,training=training)
        x = self.block4(x,training=training)
        x = self.block5(x,training=training)
        x = self.output_layer(x,training=training)
        return x
    
    def predict(self,image):
        raw_predict = self(image,training=False)
        predict = tf.argmax(raw_predict,axis=-1)
        predict = predict.numpy()
        predict = [utils.token2char(p) for p in predict]
        return predict

In [10]:
model = Model()

In [11]:
import FLAGS

FLAGS.DATA.TRAIN.TABLE_PATH = r'D:\Competitions\ComputerVision\OCR\ChinaSteel\dataset\train\overturn\cleaned_merge_table.csv'
FLAGS.DATA.TRAIN.VALIDATION_SPLIT_RATIO = 0.05
FLAGS.DATA.TRAIN.VALIDATION_SPLIT_RANDOM_STATE = 100
FLAGS.DATA.TRAIN.IMAGE_PATH = r'D:\Competitions\ComputerVision\OCR\ChinaSteel\dataset\train\overturn\image'
FLAGS.DATA.TRAIN.TRAIN_BATCH_SIZE = 4
FLAGS.DATA.TRAIN.VAL_BATCH_SIZE = 8
FLAGS.DATA.TEST.IMAGE_PATH = r'D:\Competitions\ComputerVision\OCR\ChinaSteel\dataset\test\overturn\sample'
FLAGS.DATA.TEST.BATCH_SIZE = 8

FLAGS.MODEL = model

FLAGS.LOSS.LABEL_SMOOTHING = 0.01

FLAGS.OPTIMIZER.TYPE = tf.keras.optimizers.Adam
FLAGS.OPTIMIZER.MAX_LEARNING_RATE = 1e-3
FLAGS.OPTIMIZER.SCHEDULE_GAMMA = -2

FLAGS.EPOCHS.TOTAL = 300
FLAGS.EPOCHS.WARMUP = 2

FLAGS.LOGGING.PATH = 'logs'
FLAGS.LOGGING.MODEL_NAME = 2
FLAGS.LOGGING.TRIAL_NUMBER = 1 
FLAGS.LOGGING.NOTE = ''
FLAGS.LOGGING.SAMPLES_PER_LOG = 128
FLAGS.LOGGING.TEST_IMAGE_COLUMNS = 5

In [12]:
%run train.py

KeyboardInterrupt: 